In [1]:
import requests
import numpy as np
import pandas as pd
import glob

In [3]:
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv("API_KEY")


In [4]:
for i, m in enumerate(glob.glob('./forecast_tables/forecast_[0-9][0-9][0-9][0-9].csv.gz')):
        if i == 0:
            df = pd.read_csv(m, index_col = 'Unnamed: 0')
            dates = df.date.unique()
        else:
            df = pd.concat([df, pd.read_csv(m, index_col = 'Unnamed: 0')])
            
df

,date,lower_2_5,lower_25,forecast,upper_75,upper_97_5,macroregion,prob_high,prob_low,HT,LT,HTinc,LTinc
0,2024-05-05,4276.373670,5465.242687,6209.303324,7543.832963,10095.241303,3103,99.0,0.0,3280.718607,673.196714,49.090948,10.073362
1,2024-05-12,2457.590678,3745.685553,4339.173626,5132.682524,8473.411287,3103,97.0,0.0,2544.982887,570.514218,38.081786,8.536875
2,2024-05-19,2509.969844,3711.559683,4295.637651,5264.572726,7467.471999,3103,99.0,0.0,2001.310317,532.280058,29.946555,7.964759
3,2024-05-26,2689.863188,3702.952741,4396.027439,5151.970963,7472.739677,3103,100.0,0.0,1488.280761,494.161677,22.269851,7.394375


In [5]:
df.macroregion.unique().shape

(1,)

In [6]:
df.columns

Index(['date', 'lower_2_5', 'lower_25', 'forecast', 'upper_75', 'upper_97_5',
       'macroregion', 'prob_high', 'prob_low', 'HT', 'LT', 'HTinc', 'LTinc'],
      dtype='object')

In [7]:
def post_prob_res_forecast(data: dict):
    url = "https://api.mosqlimate.org/api/vis/results-prob-forecast/"
    headers = {"X-UID-Key":API_KEY }
    return requests.post(url, json=data, headers=headers)

In [8]:
%%time
for _, r in df.iterrows(): 
    
    data = {}
    
    data["date"] = r['date']
    data['disease'] = 'dengue'
    data["geocode"] = r['macroregion']
    data["lower_2_5"] = r['lower_2_5']
    data["lower_25"] = r['lower_25']
    data["forecast"] = r['forecast']
    data["upper_75"] = r['upper_75']
    data["upper_97_5"] = r['upper_97_5']
    data["prob_high"] = r['prob_high']
    data["prob_low"] =  r['prob_low']
    data["high_threshold"] = r['HT']
    data["low_threshold"] = r['LT']
    data["high_incidence_threshold"] = r['HTinc']
    data["low_incidence_threshold"] = r['LTinc']
    
    if post_prob_res_forecast(data).status_code != 201:
        if post_prob_res_forecast(data).status_code == 403:
            pass
        else: 
            raise ValueError("Not inserted")


CPU times: user 251 ms, sys: 50.3 ms, total: 301 ms
Wall time: 7.85 s
